In [66]:
import pandas as pd
import numpy as np
import os
from typing import Callable # this is to define test_func functions dtype as function itself
import re # for regural expression
import warnings
warnings.filterwarnings('ignore')
import pickle # for pickling my model

# Can import the below dataset from 
/kaggle/input/mobile-recommendation-system-dataset/mobile_recommendation_system_dataset.csv

In [67]:
base_path = os.getcwd()
sub_path = 'data'
dataset_path = os.path.join(base_path, sub_path, 'mobile_recommendation_system_dataset.csv')

In [68]:
df = pd.read_csv(dataset_path,index_col=False)

In [69]:
df.head()

,name,ratings,price,imgURL,corpus
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...


In [70]:
def random_values(col_name:str,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[col_name].iloc[index])        

In [71]:
random_values('corpus')

2039 : Storage128 GBRAM8 GBExpandable Storage1TB  SystemAndroid 12Processor TypeG37Processor Speed2.3 50MP 50MP 5MP 5MP 2G Capacity5000 Display Size16.76 cm (6.6 inch)Resolution1612 x 720 PixelsResolution TypeHD+Display TypeHD+ IPS DisplayOther Display FeaturesRefresh Rate: 90Hz, Touch Sampling Rate: 180Hz, Brightness: Upto 500 Nits, Color Contrast Ratio: 1500:1, Panda Glass Protection, Eye Care Mode
306 : Storage128  SystemiOS 14.2Processor TypeA13 Bionic Chip 12MP 12MP 12MP 64MP 12MP 12MP 4G notMentioned Display Size15.49 cm (6.1 inch)Resolution1792 x 828 PixelsResolution TypeLiquid Retina HD DisplayHD Game SupportYesOther Display Features1400:1 Contrast Ratio (Typical), True Tone Display, Wide Colour Display (P3), Haptic Touch, 625 nits Max Brightness (Typical), Fingerprint Resistant Oleophobic Coating, Support for Display of Multiple Languages and Characters Simultaneously
2468 : Storage128 GBRAM6 GBExpandable Storage1TB  SystemAndroid 11Processor Speed2 48MP 5MP 2MP 2MP 48MP 5MP 2

## Lowercasing the corpus values

In [72]:
df['corpus'] = df['corpus'].str.lower()

In [73]:
random_values('corpus')

927 : storage64 gbram2 gbexpandable storage256gb  systemandroid 11 (go edition)processor typemediatek helio a25processor speed1.8 16mp 8mp 4g capacity5000 display size16.56 cm (6.52 inch)resolution720 x 1600$$pixelresolution typehd+gpupowervr ge8320display typeips lcd
1245 : storage32 gbram3 gbexpandable storage128gb  systemandroid oreo 8.1.0processor typemtk6739 64 bit quad coreprocessor speed1.5 13mp 8mp 3g capacity3050 display size13.84 cm (5.45 inch)resolution720 x 1440 pixelsresolution typehd+display typeips tftother display featuresfull view display, hd ips gff touch panel
1778 : storage128 gbram8  systemandroid 12primary clock speed2.4 mhzsecondary clock speed2 ghz 50mp 4g capacity6000 display size17.27 cm (6.8 inch)resolution1080x2460$$pixels
1747 : storage128 gbram8  systemandroid android 13processor speed2.2 64mp 5g capacity5000 display size16.33 cm (6.43 inch)resolution2400x1080
1009 : storage128 gbram6  systemandroid q 11processor speed90 108mp 4g capacity5000 display size1

In [74]:
df.isnull().sum()

name        0
ratings     0
price       0
imgURL      0
corpus     12
dtype: int64

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2546 entries, 0 to 2545
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     2546 non-null   object 
 1   ratings  2546 non-null   float64
 2   price    2546 non-null   object 
 3   imgURL   2546 non-null   object 
 4   corpus   2534 non-null   object 
dtypes: float64(1), object(4)
memory usage: 99.6+ KB


In [76]:
df.dropna(inplace=True)

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2534 entries, 0 to 2545
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     2534 non-null   object 
 1   ratings  2534 non-null   float64
 2   price    2534 non-null   object 
 3   imgURL   2534 non-null   object 
 4   corpus   2534 non-null   object 
dtypes: float64(1), object(4)
memory usage: 118.8+ KB


In [78]:
max_len = 0
for i in range(0, len(df['corpus'])):
    try:
        if len(df['corpus'][i]) > max_len:
            max_len = len(df['corpus'][i])
    except:
        pass

print(f'words max len in corpus is: {max_len}')


words max len in corpus is: 1196


In [79]:
df.sample(10)

,name,ratings,price,imgURL,corpus
1645,"vivo Y16 (Drizzling Gold, 128 GB)",4.3,16999,https://rukminim2.flixcart.com/image/312/312/x...,storage128 gbram4 gbexpandable storage1tb sys...
2515,SAMSUNG Galaxy M34 5G without charger (Prism S...,3.7,21299,https://rukminim2.flixcart.com/image/312/312/x...,storage128 gbram6 systemandroid android 13pro...
1160,"SAMSUNG Galaxy M21 (Midnight Blue, 128 GB)",4.1,16499,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram6 gbexpandable storage512gb s...
452,"vivo Y33s (Midday Dream, 128 GB)",4.2,15249,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram8 gbexpandable storage1tb sys...
398,"SAMSUNG Galaxy S23 5G (Lavender, 256 GB)",4.5,14463,https://rukminim2.flixcart.com/image/312/312/x...,storage256 gbram8 systemandroid 13processor t...
222,"vivo V25 5G (Surfing Blue, 128 GB)",4.3,25990,https://rukminim2.flixcart.com/image/312/312/x...,storage128 gbram8 systemandroid 12processor t...
723,"APPLE iPhone 13 mini (Starlight, 128 GB)",4.5,3537,https://rukminim2.flixcart.com/image/312/312/k...,storage128 systemios 15processor typea15 bion...
1801,"OnePlus 8T 5G (Lunar Silver, 128 GB)",4.4,29990,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram8 systemandroid q 11processor...
308,"vivo Y22 (Metaverse Green, 64 GB)",4.3,14499,https://rukminim2.flixcart.com/image/312/312/x...,storage64 gbram4 gbexpandable storage1tb syst...
526,"SAMSUNG Galaxy M21 (Midnight Blue, 64 GB)",4.2,14248,https://rukminim2.flixcart.com/image/312/312/k...,storage64 gbram4 gbexpandable storage512gb sy...


In [80]:
random_values('imgURL')

1725 : https://rukminim2.flixcart.com/image/312/312/kkyc9zk0/mobile/9/9/b/smart-5-x688c-infinix-original-imagy6wkafwrgj9z.jpeg?q=70
2509 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/m/r/i/note-12-5g-mzb0ei8in-redmi-original-imagpgr9gpmjjnwa.jpeg?q=70
1930 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/2/p/g/-original-imagkn6dk4pbcf4d.jpeg?q=70
2501 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/w/t/r/-original-imaggj68cgtdacxn.jpeg?q=70
29 : https://rukminim2.flixcart.com/image/312/312/kura1e80/mobile/r/g/a/y3s-pd2074f-bf-cf-ex-v2044-vivo-original-imag7t9mqqng2avb.jpeg?q=70
2319 : https://rukminim2.flixcart.com/image/312/312/kmthz0w0pkrrdj/mobile-refurbished/7/d/1/iphone-5c-32-a-na-apple-original-imadwyy4yvrgbyum.jpeg?q=70


# Model

## vectorization(to convert the words to vectors)   (BOW)

In [81]:
from sklearn.feature_extraction.text import CountVectorizer 

In [82]:
cont_vect = CountVectorizer(max_features=1000)

In [18]:
arrays = cont_vect.fit_transform(df['corpus'])

In [19]:
vectors = arrays.toarray()

In [20]:
cont_vect.get_feature_names_out()

array(['00', '000', '02', '03', '04', '05', '07', '08', '09', '0mp',
       '0primary', '0processor', '10', '100', '1000', '100000', '1000000',
       '100mp', '103', '104', '106', '1080', '1080processor',
       '1080resolution', '108mp', '10mp', '10processor', '11', '1100',
       '1125', '1170', '1179', '11primary', '11processor', '12', '120',
       '1200', '1200processor', '120hz', '1242', '1280processor', '1284',
       '1290', '12mp', '12primary', '12processor', '13', '1300', '1330',
       '1334', '135', '13mp', '13processor', '14', '1400', '144', '1440',
       '144hz', '1480', '14processor', '15', '1500', '1544',
       '15processor', '16', '160', '1600', '16000', '1600resolution',
       '1612', '16mp', '16processor', '18', '180', '1800', '180hz', '19',
       '1920', '1920hz', '1processor', '20', '200', '2000000', '200mp',
       '20mp', '21', '22', '23', '2340', '24', '240', '2400',
       '2400resolution', '2408', '240hz', '2460', '24mp', '25mm', '25mp',
       '26', '27'

## TFIDF to create the vector

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
TFIDF = TfidfVectorizer(max_features = 100, ngram_range = (2,2)) #here binary BOW is applied

In [40]:
vectors = TFIDF.fit_transform(df['corpus']).toarray()

In [41]:
vectors[0]

array([0.        , 0.16679359, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.26334773, 0.        ,
       0.        , 0.        , 0.        , 0.2959421 , 0.        ,
       0.23352892, 0.        , 0.        , 0.2269421 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.14455869, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.12099622, 0.        , 0.18371565, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.53714322, 0.        , 0.        , 0.        , 0.20151362,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.16441008, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.20212233, 0.        , 0.        , 0.        , 0.     

# WORD2VEC
### This creates the vector of each and every word not for the whose sentences like BOW and TFIDF

In [54]:
!pip install gensim


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [178]:
df.index

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       2535, 2536, 2537, 2538, 2539, 2540, 2541, 2542, 2544, 2545],
      dtype='int64', length=2534)

In [212]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
corpus = []
for i in df.index:
    description = df['corpus'][i]
    description = description.split()
    
    description = [lemmatizer.lemmatize(word) for word in description]
    description = ' '.join(description)
    corpus.append(description)
    

In [213]:
corpus

['storage128 gbram6 systemandroid 12processor typemediatek dimensity 1080processor speed2.6 50mp 8mp 2mp 50mp 16mp 5g capacity5000 display size16.94 cm (6.67 inch)resolution2400 x 1080 pixelsresolution typefull hd+ amoled displaygpuarm mali-g68 mc4display typefull hd+ amoled displayother display featuresrefresh rate: 120 hz, contrast: 5,000,000:1, 1920hz pwm dimming, brightness level: 16000:1, peak brightness: 900 nit',
 'storage128 gbram6 gbexpandable storage256gb systemandroid pie 9.0processor typemediatek helio p70 octa core 2.1 ghzprocessor speed2.1 48mp 5mp 48mp 5mp 48mp 5mp 48mp 5mp 48mp 5mp 16mp 3g capacity4000 display size16.51 cm (6.5 inch)resolution2340 x 1080 pixelsresolution typefull hd+gpuarm mali-g72 mp3 at 900 mhzdisplay colors16mother display featuresscreen texture: tft-ltps, in-cell touch panel technology, narrow frame: 1.70mm (size from aa to outside edge of glass), screen ratio: 90.90%, screen contrast: 1500:1 (typical), color saturation: ntsc 81.5% (typical), maximu

In [214]:
from nltk import sent_tokenize
# from gensim.utils import simple_preprocess

In [215]:
corpus[0]

'storage128 gbram6 systemandroid 12processor typemediatek dimensity 1080processor speed2.6 50mp 8mp 2mp 50mp 16mp 5g capacity5000 display size16.94 cm (6.67 inch)resolution2400 x 1080 pixelsresolution typefull hd+ amoled displaygpuarm mali-g68 mc4display typefull hd+ amoled displayother display featuresrefresh rate: 120 hz, contrast: 5,000,000:1, 1920hz pwm dimming, brightness level: 16000:1, peak brightness: 900 nit'

In [216]:
simple_preprocess(corpus[0])

['storage',
 'gbram',
 'systemandroid',
 'processor',
 'typemediatek',
 'dimensity',
 'processor',
 'speed',
 'mp',
 'mp',
 'mp',
 'mp',
 'mp',
 'capacity',
 'display',
 'size',
 'cm',
 'inch',
 'resolution',
 'typefull',
 'hd',
 'amoled',
 'displaygpuarm',
 'mali',
 'mc',
 'display',
 'typefull',
 'hd',
 'amoled',
 'displayother',
 'display',
 'featuresrefresh',
 'rate',
 'hz',
 'contrast',
 'hz',
 'pwm',
 'dimming',
 'brightness',
 'level',
 'peak',
 'brightness',
 'nit']

In [217]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(word_tokenize(sent))

In [218]:
words[0]

['storage128',
 'gbram6',
 'systemandroid',
 '12processor',
 'typemediatek',
 'dimensity',
 '1080processor',
 'speed2.6',
 '50mp',
 '8mp',
 '2mp',
 '50mp',
 '16mp',
 '5g',
 'capacity5000',
 'display',
 'size16.94',
 'cm',
 '(',
 '6.67',
 'inch',
 ')',
 'resolution2400',
 'x',
 '1080',
 'pixelsresolution',
 'typefull',
 'hd+',
 'amoled',
 'displaygpuarm',
 'mali-g68',
 'mc4display',
 'typefull',
 'hd+',
 'amoled',
 'displayother',
 'display',
 'featuresrefresh',
 'rate',
 ':',
 '120',
 'hz',
 ',',
 'contrast',
 ':',
 '5,000,000:1',
 ',',
 '1920hz',
 'pwm',
 'dimming',
 ',',
 'brightness',
 'level',
 ':',
 '16000:1',
 ',',
 'peak',
 'brightness',
 ':',
 '900',
 'nit']

In [219]:
from gensim.models import Word2Vec


# Train a Word2Vec model7
model = Word2Vec(words, vector_size=100, window=5, min_count=1, workers=4)

In [220]:
model.wv['dimensity']

array([ 0.20212714, -0.65989184,  0.7894744 ,  0.49276662, -0.388531  ,
       -0.8446683 , -0.37368575,  0.37646812,  0.7125481 , -0.73005056,
       -0.4990519 ,  0.6629855 , -0.02820136,  0.56469405,  0.16870205,
       -0.47449332,  0.45296168, -1.0438422 ,  0.5622541 , -0.5570093 ,
       -0.32614267,  1.3406439 , -0.9827848 , -0.5575917 ,  0.49339756,
        0.06724337, -1.0113013 ,  0.4703061 ,  0.18993285,  0.8675029 ,
        0.38140097,  0.8901985 , -0.43939644, -1.0250883 , -1.1083192 ,
        0.5650256 , -0.24270847,  0.11534074, -0.7835607 , -0.7402529 ,
        0.5170808 ,  0.73933595, -0.5180775 ,  0.01925875, -0.23575021,
       -0.9875561 , -0.09335922, -0.15175366,  0.0764869 ,  0.22074182,
        0.59230906,  0.27185938,  0.6043667 , -1.1543534 , -0.2384987 ,
        0.3237121 , -0.7772877 , -0.11115357, -0.21285474, -0.25382861,
       -0.40541512,  0.5767194 , -0.32693064, -0.7224819 , -0.0561279 ,
        0.5644377 , -0.545228  ,  0.07198086, -1.1668732 , -0.39

In [221]:
model.wv.index_to_key

[',',
 'display',
 ')',
 '(',
 ':',
 'inch',
 'cm',
 '2mp',
 'x',
 'systemandroid',
 'ratio',
 '12mp',
 '8mp',
 'pixelsresolution',
 'typefull',
 '%',
 'brightness',
 'gbexpandable',
 '1080',
 '4g',
 'capacity5000',
 '50mp',
 'storage128',
 '5mp',
 'rate',
 '13mp',
 'screen',
 '5g',
 'contrast',
 'touch',
 'hd+',
 '48mp',
 'nit',
 '16mp',
 'hz',
 '$',
 'color',
 'storage64',
 'displayother',
 'typical',
 'gbram8',
 'typemediatek',
 'retina',
 'gbram4',
 '11processor',
 'typequalcomm',
 '12processor',
 'lcd',
 'displaydisplay',
 'snapdragon',
 '64mp',
 'amoled',
 'glass',
 'refresh',
 'resolution2400',
 'xdr',
 '720',
 'typesuper',
 'max',
 'wide',
 'aspect',
 'speed2',
 '32mp',
 '3g',
 'gbram6',
 'screen-to-body',
 'support',
 'and',
 'storage256gb',
 'coating',
 'storage256',
 'oleophobic',
 'helio',
 'storage1tb',
 'systemios',
 'pixel',
 'speed2.4',
 'p3',
 'ntsc',
 'chip',
 'for',
 'bionic',
 'game',
 'nits',
 'hdr',
 'typehd+',
 'mode',
 'sampling',
 '20:9',
 'ip',
 '-',
 'of',
 '

In [222]:
def avg_word2vec(doc):
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [223]:
from tqdm import tqdm

In [224]:
words[0]

['storage128',
 'gbram6',
 'systemandroid',
 '12processor',
 'typemediatek',
 'dimensity',
 '1080processor',
 'speed2.6',
 '50mp',
 '8mp',
 '2mp',
 '50mp',
 '16mp',
 '5g',
 'capacity5000',
 'display',
 'size16.94',
 'cm',
 '(',
 '6.67',
 'inch',
 ')',
 'resolution2400',
 'x',
 '1080',
 'pixelsresolution',
 'typefull',
 'hd+',
 'amoled',
 'displaygpuarm',
 'mali-g68',
 'mc4display',
 'typefull',
 'hd+',
 'amoled',
 'displayother',
 'display',
 'featuresrefresh',
 'rate',
 ':',
 '120',
 'hz',
 ',',
 'contrast',
 ':',
 '5,000,000:1',
 ',',
 '1920hz',
 'pwm',
 'dimming',
 ',',
 'brightness',
 'level',
 ':',
 '16000:1',
 ',',
 'peak',
 'brightness',
 ':',
 '900',
 'nit']

In [225]:
#apply for the entire sentences
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

100%|██████████| 2573/2573 [00:01<00:00, 2364.02it/s]


In [226]:
X[0]

array([-0.0111495 ,  0.18098125,  0.68456864,  0.15131907, -0.32766125,
       -0.23051241, -0.03370231,  0.6700672 , -0.43438548, -0.3725086 ,
        0.0555828 , -0.23332267, -0.1512541 ,  0.38709697, -0.24058641,
       -0.36759678,  0.36836767, -0.1821268 , -0.4564861 , -0.77440584,
        0.30407795, -0.07565396,  0.34940282, -0.1409268 , -0.38050714,
        0.07471731,  0.02741507, -0.41477385,  0.00651333,  0.35908407,
        0.58441466, -0.27505696,  0.30358922, -0.5940229 , -0.3568502 ,
       -0.15418701,  0.2276752 ,  0.06775934,  0.28720355, -0.7268598 ,
        0.05929705, -0.5839155 ,  0.21071061,  0.27710286,  0.3220522 ,
       -0.30329305, -0.19497883, -0.29049438,  0.34198046,  0.02827258,
       -0.34046403, -0.24924318, -0.17160636, -0.23919958, -0.42113104,
        0.08283345,  0.31873077,  0.0034126 , -0.06700233,  0.2422882 ,
        0.07322871,  0.23728591,  0.4370739 ,  0.42763764, -0.50853896,
        0.38382667, -0.10128225,  0.5051837 , -0.5960318 ,  0.25

## Calculating cosine similarity for recommendation

In [227]:
from sklearn.metrics.pairwise import cosine_similarity

In [228]:
# similarity = cosine_similarity(vectors)
similarity = cosine_similarity(X)
similarity.shape # this is because I have calculated the similarity among all the rows to all the rows(confusing?? but try to understand)

(2573, 2573)

In [229]:
similarity[2567]

array([0.64117336, 0.31615785, 0.98400044, ..., 0.9650653 , 0.89074916,
       0.9815264 ], dtype=float32)

In [230]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:11]

[(68, 1.0000002),
 (303, 1.0000002),
 (1489, 0.999931),
 (634, 0.99993086),
 (466, 0.9998522),
 (1045, 0.9998522),
 (1763, 0.9998522),
 (587, 0.99606365),
 (1141, 0.99521464),
 (1568, 0.9952146)]

In [231]:
def recommend(mobile):
    mobile_index = df[df['name']==mobile].index[0]
    similarity_array = similarity[mobile_index]
    similar_10_mobiles = sorted(list(enumerate(similarity_array)),reverse=True,key=lambda x:x[1])[1:11]
    
    for i in similar_10_mobiles:
        print(df.iloc[i[0]])
        # print(df['corpus'].iloc[i[0]])



In [232]:
df['name'][0:10]

0      REDMI Note 12 Pro 5G (Onyx Black, 128 GB)
1            OPPO F11 Pro (Aurora Green, 128 GB)
2         REDMI Note 11 (Starburst White, 64 GB)
3         OnePlus Nord CE 5G (Blue Void, 256 GB)
4            APPLE iPhone 13 mini (Blue, 128 GB)
5             APPLE iPhone 6s Plus (Gold, 64 GB)
6         REDMI 10 Power (Sporty Orange, 128 GB)
7          Infinix Hot 11 (Emerald Green, 64 GB)
8    Infinix Note 10 Pro (Nordic Secret, 256 GB)
9             SAMSUNG Galaxy A04 (Green, 128 GB)
Name: name, dtype: object

In [233]:
df.columns

Index(['name', 'ratings', 'price', 'imgURL', 'corpus'], dtype='object')

In [235]:
recommend('REDMI Note 12 Pro 5G (Onyx Black, 128 GB)')

name          REDMI Note 12 Pro 5G (Stardust Purple, 128 GB)
ratings                                                  4.2
price                                                  23999
imgURL     https://rukminim2.flixcart.com/image/312/312/x...
corpus     storage128 gbram6  systemandroid 12processor t...
Name: 69, dtype: object
name             REDMI Note 12 Pro 5G (Glacier Blue, 128 GB)
ratings                                                  4.2
price                                                  23999
imgURL     https://rukminim2.flixcart.com/image/312/312/x...
corpus     storage128 gbram6  systemandroid 12processor t...
Name: 305, dtype: object
name                 SAMSUNG Galaxy F62 (Laser Grey, 128 GB)
ratings                                                  4.0
price                                                  25999
imgURL     https://rukminim2.flixcart.com/image/312/312/k...
corpus     storage128 gbram8 gbexpandable storage1tb  sys...
Name: 1493, dtype: object
name      

# Its is good to see that word2vec is working much better.

## making Pickle file of df and similarities to use in web

In [77]:
os.makedirs(r'src\model',exist_ok=True)

In [78]:
# pickle.dump(obj=df,file=open(file=r'src\model\dataframe.pkl',mode='wb'))

In [236]:
pickle.dump(obj=similarity,file=open(file=r'src\model\similarity.pkl',mode='wb'))

### Ok, till now what we did is if the user gives the exact same name of the product then we fetch the corresponding corpus and then we find the vectors which are similar to it and then we fetch those similar corpus vectors corresponding name and gives as recommendation.

### now the problem is that user when enter in search bar will not enter the exact same name that of in the table. so we cannot get the corresponding corpus, eventually leads to error.
### So, When user enters for eg. Iphone 13, then by using some searching model some recommendations should be given based on Names itself, and then when the user click on the any recommended product then we will have the proper name of the product for recommendation then we can use the model that we created till now.
### Now, we need to create the model to give recommendation on giving the name.

In [135]:
df.columns

Index(['name', 'ratings', 'price', 'imgURL', 'corpus'], dtype='object')

In [136]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
names = []
for i in df.index:
    name = df['name'][i].lower()
    name = name.split()
    # Remove unwanted characters
    name = [re.sub(r'[()\[\],]', '', word) for word in name if re.sub(r'[()\[\],]', '', word) != '']

    name = [lemmatizer.lemmatize(word) for word in name]
    name = ' '.join(name)
    names.append(name)
    

In [137]:
from nltk import sent_tokenize, word_tokenize
from gensim.utils import simple_preprocess

In [138]:
name_words=[]
for sent in names:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        name_words.append(word_tokenize(sent))

In [139]:
df.iloc[0]

name               REDMI Note 12 Pro 5G (Onyx Black, 128 GB)
ratings                                                  4.2
price                                                  23999
imgURL     https://rukminim2.flixcart.com/image/312/312/x...
corpus     storage128 gbram6  systemandroid 12processor t...
Name: 0, dtype: object

In [140]:
name_words[0]

['redmi', 'note', '12', 'pro', '5g', 'onyx', 'black', '128', 'gb']

In [141]:
name_words

[['redmi', 'note', '12', 'pro', '5g', 'onyx', 'black', '128', 'gb'],
 ['oppo', 'f11', 'pro', 'aurora', 'green', '128', 'gb'],
 ['redmi', 'note', '11', 'starburst', 'white', '64', 'gb'],
 ['oneplus', 'nord', 'ce', '5g', 'blue', 'void', '256', 'gb'],
 ['apple', 'iphone', '13', 'mini', 'blue', '128', 'gb'],
 ['apple', 'iphone', '6', 'plus', 'gold', '64', 'gb'],
 ['redmi', '10', 'power', 'sporty', 'orange', '128', 'gb'],
 ['infinix', 'hot', '11', 'emerald', 'green', '64', 'gb'],
 ['infinix', 'note', '10', 'pro', 'nordic', 'secret', '256', 'gb'],
 ['samsung', 'galaxy', 'a04', 'green', '128', 'gb'],
 ['vivo', 'z1pro', 'sonic', 'black', '64', 'gb'],
 ['moto', 'c', 'plus', 'pearl', 'white', '16', 'gb'],
 ['oppo', 'a15', 'dynamic', 'black', '32', 'gb'],
 ['samsung', 'galaxy', 'z', 'flip4', '5g', 'bora', 'purple', '128', 'gb'],
 ['realme', 'gt', '2', 'pro', 'paper', 'green', '256', 'gb'],
 ['oneplus', '9', 'pro', '5g', 'morning', 'mist', '256', 'gb'],
 ['apple', 'iphone', '14', 'plus', 'productr

In [142]:
from gensim.models import Word2Vec

# Train a Word2Vec model
name_model = Word2Vec(name_words, vector_size=100, window=5, min_count=1, workers=4)

In [143]:
name_model.wv.index_to_key

['gb',
 '128',
 '64',
 '5g',
 'black',
 'blue',
 'samsung',
 'galaxy',
 '256',
 'apple',
 'iphone',
 'pro',
 'redmi',
 'vivo',
 '32',
 'oppo',
 'note',
 'green',
 'realme',
 'white',
 'gold',
 'oneplus',
 'infinix',
 'tecno',
 'silver',
 '11',
 '12',
 'grey',
 '10',
 'plus',
 'spark',
 '7',
 'motorola',
 'purple',
 '6',
 'red',
 'max',
 '8',
 '9',
 '16',
 '13',
 'nord',
 'midnight',
 '512',
 'hot',
 'prime',
 '14',
 'awesome',
 'space',
 '2',
 'power',
 'gray',
 'mi',
 'phantom',
 'xiaomi',
 '5',
 'mini',
 '3',
 'se',
 'smart',
 'lite',
 'xr',
 'cosmic',
 '20',
 'ce',
 'sky',
 'glacier',
 'ice',
 'graphite',
 'includes',
 'earpods',
 'adapter',
 'edition',
 'prism',
 'diamond',
 'narzo',
 'rose',
 'reno10',
 'orange',
 'aqua',
 's22',
 'gt',
 'aurora',
 'sea',
 'starry',
 'matte',
 'go',
 'ultra',
 'mint',
 'x',
 'starlight',
 'dark',
 'coral',
 'camon',
 'm32',
 's23',
 't1',
 '30',
 'edge',
 'mirror',
 'neo',
 'cyan',
 '2023',
 'light',
 'a53',
 'violet',
 'pro+',
 'nebula',
 '10r',


In [144]:
def avg_word2vec(doc):
    return np.mean([name_model.wv[word] for word in doc if word in name_model.wv.index_to_key],axis=0)

In [165]:
len(name_words)

2535

In [145]:
from tqdm import tqdm
#apply for the entire sentences
name_vectors=[]
for i in tqdm(range(len(name_words))):
    name_vectors.append(avg_word2vec(name_words[i]))

100%|██████████| 2535/2535 [00:00<00:00, 14383.76it/s]


In [166]:
len(name_vectors)

2535

In [146]:
name_vectors[0]

array([-0.13094905,  0.202058  ,  0.14212401,  0.09102975,  0.05367602,
       -0.37493283,  0.2674204 ,  0.5999901 , -0.23589168, -0.26791218,
       -0.09167697, -0.3072817 , -0.0403779 ,  0.09827185,  0.04134549,
       -0.23755932,  0.15242456, -0.20687503, -0.03629224, -0.5499929 ,
        0.20011626,  0.11807813,  0.2114154 , -0.09873579, -0.07481906,
       -0.01852687, -0.24881972, -0.17310408, -0.13235804,  0.05497935,
        0.24908927,  0.04378024,  0.10369452, -0.4015833 , -0.19374527,
        0.35321683,  0.06774788, -0.09895633, -0.234363  , -0.35146877,
        0.11906157, -0.2061286 , -0.08995955,  0.05555212,  0.22768916,
       -0.15969329, -0.17089124, -0.11924712,  0.20901611,  0.17184657,
        0.11877979, -0.09525224, -0.03208587, -0.12169465, -0.07047401,
       -0.00442161,  0.18184003, -0.07263005, -0.21176833,  0.09565994,
        0.08032079, -0.07440865,  0.16528316, -0.07992143, -0.21607536,
        0.18695104,  0.154527  ,  0.39495233, -0.33316812,  0.16

In [147]:
name_vectors

[array([-0.13094905,  0.202058  ,  0.14212401,  0.09102975,  0.05367602,
        -0.37493283,  0.2674204 ,  0.5999901 , -0.23589168, -0.26791218,
        -0.09167697, -0.3072817 , -0.0403779 ,  0.09827185,  0.04134549,
        -0.23755932,  0.15242456, -0.20687503, -0.03629224, -0.5499929 ,
         0.20011626,  0.11807813,  0.2114154 , -0.09873579, -0.07481906,
        -0.01852687, -0.24881972, -0.17310408, -0.13235804,  0.05497935,
         0.24908927,  0.04378024,  0.10369452, -0.4015833 , -0.19374527,
         0.35321683,  0.06774788, -0.09895633, -0.234363  , -0.35146877,
         0.11906157, -0.2061286 , -0.08995955,  0.05555212,  0.22768916,
        -0.15969329, -0.17089124, -0.11924712,  0.20901611,  0.17184657,
         0.11877979, -0.09525224, -0.03208587, -0.12169465, -0.07047401,
        -0.00442161,  0.18184003, -0.07263005, -0.21176833,  0.09565994,
         0.08032079, -0.07440865,  0.16528316, -0.07992143, -0.21607536,
         0.18695104,  0.154527  ,  0.39495233, -0.3

In [148]:
pickle.dump(obj=name_vectors,file=open(file=r'src\model\name_vectors.pkl',mode='wb'))

In [149]:
# Example: Filter out irregular vectors
problem_vectors = [[i, name_vectors[i]] for i in range(len(name_vectors)) if type(name_vectors[i]) == np.float64]

In [150]:
problem_vectors

[]

In [151]:
# name_vectors.pop(2433)


# Run from here

In [237]:
import pickle
# Load the saved model
with open(r'src\model\similarity.pkl', 'rb') as file:
    similarity = pickle.load(file)

with open(r'src\model\name_model.pkl', 'rb') as file:
    name_model = pickle.load(file)

with open(r'src\model\name_vectors.pkl', 'rb') as file:
    name_vectors = pickle.load(file)

In [238]:
similarity.shape

(2573, 2573)

In [239]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
from nltk import sent_tokenize, word_tokenize
import numpy as np

In [240]:
def avg_word2vec(doc):
    return np.mean([name_model.wv[word] for word in doc if word in name_model.wv.index_to_key],axis=0)

In [241]:
from nltk.tokenize import word_tokenize
input_string = input().lower()
input_string = input_string.split()
input_string = [lemmatizer.lemmatize(word) for word in input_string]
input_string = ' '.join(input_string)
print(input_string)
sent_token=sent_tokenize(input_string)
print(sent_token)
sim_pro = word_tokenize(sent_token[0])
print(sim_pro)
w2v = avg_word2vec(sim_pro)
print(w2v)

iphone 14 pro max
['iphone 14 pro max']
['iphone', '14', 'pro', 'max']
[-0.08658782  0.1504207   0.09215908  0.06089884  0.02871851 -0.28389156
  0.18762505  0.44933635 -0.17588404 -0.19557907 -0.06158967 -0.23170465
 -0.01680407  0.06701943  0.04152417 -0.16718376  0.11277737 -0.1589049
 -0.01931311 -0.4170555   0.13784048  0.08066157  0.14670533 -0.07581823
 -0.05996743 -0.02138675 -0.18948586 -0.13525623 -0.09901463  0.03122792
  0.18521366  0.02765223  0.08582246 -0.29058066 -0.14153787  0.26896864
  0.0587853  -0.06476524 -0.17725885 -0.262575    0.06981059 -0.14096805
 -0.07160798  0.04126026  0.17171685 -0.12867056 -0.12824391 -0.0737525
  0.15344052  0.13506544  0.08491828 -0.06206045 -0.02348918 -0.08811501
 -0.0362386  -0.0097202   0.12762098 -0.04751685 -0.15137273  0.05973624
  0.04679416 -0.05270179  0.12322068 -0.05770503 -0.16405363  0.14055878
  0.09722815  0.27910948 -0.24543825  0.11074062 -0.03470847  0.16131386
  0.15325834  0.0417752   0.27655786  0.03811543  0.032

In [242]:
w2v.shape

(100,)

In [243]:
w2v_reshaped = w2v.reshape(1,-1)

In [244]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_name_words = cosine_similarity(w2v_reshaped, name_vectors)
similarity_name_words

array([[0.998399  , 0.9985727 , 0.99837273, ..., 0.9980001 , 0.998036  ,
        0.9982687 ]], dtype=float32)

In [245]:
similar_10_mobiles = sorted(list(enumerate(similarity_name_words[0])),reverse=True,key=lambda x:x[1])[0:10]
for i in similar_10_mobiles:
    print(df['name'].iloc[i[0]])

APPLE iPhone 14 Pro Max (Gold, 1 TB)
realme C25_Y (Metal Grey, 128 GB)
APPLE iPhone 14 Pro Max (Gold, 512 GB)
APPLE iPhone 14 Pro Max (Silver, 512 GB)
OPPO F3 Plus (Gold, 64 GB)
APPLE iPhone 14 Pro Max (Deep Purple, 1 TB)
APPLE iPhone 14 Pro Max (Deep Purple, 512 GB)
APPLE iPhone 13 Pro Max (Graphite, 1 TB)
APPLE iPhone 14 Pro Max (Space Black, 1 TB)
APPLE iPhone 14 Pro (Silver, 1 TB)


In [246]:
similarity.shape

(2573, 2573)

In [247]:
def recommend(mobile):
    mobile_index = df[df['name']==mobile].index[0]
    similarity_array = similarity[mobile_index]
    similar_10_mobiles = sorted(list(enumerate(similarity_array)),reverse=True,key=lambda x:x[1])[1:11]
    print(similar_10_mobiles)
    
    for i in similar_10_mobiles:
        print(df.iloc[i[0]])


In [248]:
len(df)

2534

In [249]:
recommend('APPLE iPhone 14 Pro (Silver, 1 TB)')

[(1084, 1.0000001), (1446, 1.0000001), (2394, 1.0000001), (691, 0.99991745), (1445, 0.99991745), (2201, 0.99991745), (142, 0.9964064), (446, 0.9964064), (257, 0.9963761), (537, 0.9963761)]
name                  Infinix Smart 6 (Starry Purple, 64 GB)
ratings                                                  4.3
price                                                   6999
imgURL     https://rukminim2.flixcart.com/image/312/312/l...
corpus     storage64 gbram2 gbexpandable storage512gb  sy...
Name: 1087, dtype: object
name                    APPLE iPhone XS (Space Grey, 256 GB)
ratings                                                  4.5
price                                                  27172
imgURL     https://rukminim2.flixcart.com/image/312/312/j...
corpus     storage256  systemios 12processor typea12 bion...
Name: 1450, dtype: object
name                          REDMI A1+ (Light Green, 32 GB)
ratings                                                  4.1
price                      

In [197]:
def name_recommend(text):
    input_string = text.lower()
    input_string = input_string.split()
    input_string = [lemmatizer.lemmatize(word) for word in input_string]
    input_string = ' '.join(input_string)
    sent_token=sent_tokenize(input_string)
    sim_pro = word_tokenize(sent_token[0])
    w2v = avg_word2vec(sim_pro)
    w2v_reshaped = w2v.reshape(1,-1)
    similarity_name_words = cosine_similarity(w2v_reshaped, name_vectors)
    similar_10_mobiles = sorted(list(enumerate(similarity_name_words[0])),reverse=True,key=lambda x:x[1])[0:10]
    for i in similar_10_mobiles:
        print(df['name'].iloc[i[0]])

In [200]:
name_recommend(input())

Redmi Note 4 (Black, 64 GB)
Redmi Note 4 (Gold, 64 GB)
Redmi Note 4 (Gold, 32 GB)
Redmi Note 4 (Dark Grey, 32 GB)
Redmi Note 4 (Lake Blue, 64 GB)
Redmi Note 8 (Cosmic Purple, 64 GB)
REDMI Note 11S (Space Black, 64 GB)
REDMI Note 11 (Space Black, 64 GB)
REDMI Note 11 (Space Black, 64 GB)
Redmi Note 8 (Space Black, 64 GB)


In [ ]:
name_model.wv['apple']

array([-0.10993522,  0.15476543,  0.09424664,  0.07854822,  0.02877077,
       -0.32960406,  0.21019301,  0.5103647 , -0.20055765, -0.22535747,
       -0.06911097, -0.26333624, -0.03115895,  0.08076768,  0.04080999,
       -0.17979114,  0.1288311 , -0.18892051, -0.0158529 , -0.47468203,
        0.15784214,  0.10232893,  0.15743501, -0.08507022, -0.05232415,
       -0.02711057, -0.22233924, -0.13980141, -0.11486401,  0.02887402,
        0.22377141,  0.03877762,  0.09956583, -0.31504652, -0.17332412,
        0.30549186,  0.06838248, -0.06607033, -0.18711601, -0.28831863,
        0.08957129, -0.15398338, -0.08650946,  0.03601255,  0.19631867,
       -0.13800915, -0.15609558, -0.07266942,  0.17927726,  0.15996735,
        0.10286544, -0.05548932, -0.02651755, -0.09251658, -0.03474584,
       -0.01779863,  0.14300776, -0.05958966, -0.1655504 ,  0.06755746,
        0.05114037, -0.04799179,  0.15299432, -0.07004677, -0.18934068,
        0.15044437,  0.10403606,  0.30413938, -0.2734699 ,  0.12

In [203]:
pickle.dump(obj=name_model,file=open(file=r'src\model\name_model.pkl',mode='wb'))

{'first': 0, 'second': 0}


In [4]:
st = 'trayA1'
print(st[:-1])

trayA
